In [1]:
import pandas as pd
import json

In [6]:
# Create a DataFrame from the JSON file
path = 'resources/neo_data.json'

df = pd.read_json(path)
df.info()

,id,absolute_magnitude_h,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,sentry_object,is_potentially_hazardous
count,8.218000e+03,8218.000000,8218.000000,8218.000000,8218.000000,8.218000e+03,8218.000000,8218.000000
mean,3.513919e+07,24.110806,0.092469,0.206766,45562.511475,3.185755e+07,0.055853,0.067170
std,2.469146e+07,2.726174,0.173444,0.387832,24427.372079,2.329364e+07,0.229652,0.250331
min,2.001685e+06,14.280000,0.000818,0.001828,1198.066272,1.036104e+04,0.000000,0.000000
25%,3.761397e+06,22.250000,0.016016,0.035813,27365.635535,9.111011e+06,0.000000,0.000000
50%,5.426826e+07,24.340000,0.036021,0.080545,40903.205684,2.984113e+07,0.000000,0.000000
75%,5.439746e+07,26.100000,0.094309,0.210882,59549.357704,5.200220e+07,0.000000,0.000000
max,5.445374e+07,32.560000,3.703011,8.280184,163244.185025,7.478480e+07,1.000000,1.000000
